In [23]:
%pip install langchain pyarrow chromadb tiktoken tabulate sqlalchemy sqlalchemy-bigquery google-cloud-aiplatform google-cloud-bigquery google-cloud-bigquery-storage

E0803 07:43:14.276572924    2389 backup_poller.cc:127]                 Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2023-08-03T07:43:14.275933892+00:00", children:[UNKNOWN:Bad file descriptor {created_time:"2023-08-03T07:43:14.275802926+00:00", errno:9, os_error:"Bad file descriptor", syscall:"epoll_wait"}]}

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!sudo apt install sqlite3

需要配置 GOOGLE_APPLICATION_CREDENTIALS 环境变量
export GOOGLE_APPLICATION_CREDENTIALS=/home/user/code/sa.json

In [1]:
!export GOOGLE_APPLICATION_CREDENTIALS=/home/user/code/sa.json

重启 Kernel

In [31]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}

: 

测试是否可以调用Palm2 API

In [1]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = VertexAI()

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)

'Justin Beiber was born on March 1, 1994. The Super Bowl in 1994 was won by the San Francisco 49ers.\nThe final answer: San Francisco 49ers.'

测试LLM with SQL

使用Sqlite, 
数据库来源 https://database.guide/2-sample-databases-sqlite/

In [2]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor

service_account_file = "/home/user/code/sa.json" # Change to where your service account key file is located
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_file

table = "sales"
sqlalchemy_url_chin = f'sqlite:///Chinbook.db'

In [3]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

llm = VertexAI()
db = SQLDatabase.from_uri(sqlalchemy_url_chin)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
llm=llm,
toolkit=toolkit,
verbose=True,
top_k=40,
)

def query(question_str):
    return agent_executor.run(question_str)


In [4]:
query("Describe the playlisttrack table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought:I now know the final answer
Final Answer: The playlisttrack table contains the track ids of the tracks in a playlist.

> Finished chain.


'The playlisttrack table contains the track ids of the tracks in a playlist.'

In [5]:
query("List the total sales per country. Which country's customers spent the most?")




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:I should look at the schema of the Invoice table to see what columns I can query.
Action: sql_db_schema
Action Input: Invoice
Observation: 
CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	2	2009-01-01 00:00:00	Theodor-Heuss-Straße 34	Stuttgart	None	Germany	70174	1.98
2	4	2009

/home/user/code/.venv/lib/python3.10/site-packages/langchain/sql_database.py:353: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows_result = connection.execute(command)  # type: ignore


I should query the Invoice table to get the total sales per country.
Action: sql_db_query
Action Input: SELECT BillingCountry, SUM(Total) FROM Invoice GROUP BY BillingCountry
Observation: [('Argentina', 37.620000000000005), ('Australia', 37.620000000000005), ('Austria', 42.62), ('Belgium', 37.62), ('Brazil', 190.09999999999997), ('Canada', 303.9599999999999), ('Chile', 46.62), ('Czech Republic', 90.24000000000001), ('Denmark', 37.620000000000005), ('Finland', 41.620000000000005), ('France', 195.09999999999994), ('Germany', 156.48), ('Hungary', 45.62), ('India', 75.25999999999999), ('Ireland', 45.62), ('Italy', 37.620000000000005), ('Netherlands', 40.62), ('Norway', 39.62), ('Poland', 37.620000000000005), ('Portugal', 77.23999999999998), ('Spain', 37.62), ('Sweden', 38.620000000000005), ('USA', 523.0600000000003), ('United Kingdom', 112.85999999999999)]
Thought:I should order the results by the total sales in descending order.
Action: sql_db_query
Action Input: SELECT BillingCountry, SU

'The USA has the highest total sales, with 523.06.'

In [36]:
query("Show the total number of tracks in each playlist. The Playlist name should be included in the result.")




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: sales
Thought:I found the table sales. I should query the schema of this table to see what columns it has.
Action: sql_db_schema
Action Input: sales
Observation: 
CREATE TABLE `sales` (
	`invoice_and_item_number` STRING, 
	`date` DATE, 
	`store_number` STRING, 
	`store_name` STRING, 
	`address` STRING, 
	`city` STRING, 
	`zip_code` STRING, 
	`county_number` STRING, 
	`county` STRING, 
	`category` STRING, 
	`category_name` STRING, 
	`vendor_number` STRING, 
	`vendor_name` STRING, 
	`item_number` STRING, 
	`item_description` STRING, 
	`pack` INT64, 
	`bottle_volume_ml` INT64, 
	`state_bottle_cost` FLOAT64, 
	`state_bottle_retail` FLOAT64, 
	`bottles_sold` INT64, 
	`sale_dollars` FLOAT64, 
	`volume_sold_liters` FLOAT64, 
	`volume_sold_gallons` FLOAT64
)

/*
3 rows from sales table:
invoice_and_item_number	date	store_number	store_name	address	city	zip_code	county_number	county	category	category_n

'The stores with the highest number of sales are:\n1. HY-VEE FOOD STORE / URBANDALE, zip code 50322, sale dollars 279557.28, city URBANDALE\n2. HY-VEE #3 / BDI / DES MOINES, zip code 50320, sale dollars 254100.0, city DES MOINES\n3. HY-VEE #3 / BDI / DES MOINES, zip code'



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: sales
Thought:The sales table seems relevant. I should query the schema of the sales table.
Action: sql_db_schema
Action Input: sales
Observation: 
CREATE TABLE `sales` (
	`invoice_and_item_number` STRING, 
	`date` DATE, 
	`store_number` STRING, 
	`store_name` STRING, 
	`address` STRING, 
	`city` STRING, 
	`zip_code` STRING, 
	`county_number` STRING, 
	`county` STRING, 
	`category` STRING, 
	`category_name` STRING, 
	`vendor_number` STRING, 
	`vendor_name` STRING, 
	`item_number` STRING, 
	`item_description` STRING, 
	`pack` INT64, 
	`bottle_volume_ml` INT64, 
	`state_bottle_cost` FLOAT64, 
	`state_bottle_retail` FLOAT64, 
	`bottles_sold` INT64, 
	`sale_dollars` FLOAT64, 
	`volume_sold_liters` FLOAT64, 
	`volume_sold_gallons` FLOAT64
)

/*
3 rows from sales table:
invoice_and_item_number	date	store_number	store_name	address	city	zip_code	county_number	county	category	category_name	vendor_numb

In [ ]:
%pip install gradio

In [6]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Ask BigQuery

    This demo is to showcase answering questions on a tabular data available in Big Query using Vertex PALM LLM & Langchain.

    Dataset: Public dataset on Iowa Liquor Sales

    Dataset dimensions: ~26 million rows and 24 columns



    ### Sample Inputs:
    1. Show the total number of tracks in each playlist. The Playlist name should be included in the result.
    ### Enter a search query...

    """)
    with gr.Row():
      with gr.Column():
        input_text = gr.Textbox(label="Question", placeholder="Show the total number of tracks in each playlist. The Playlist name should be included in the result.")

    with gr.Row():
      generate = gr.Button("Ask SQL")

    with gr.Row():
      label2 = gr.Textbox(label="Output")
    # with gr.Row():
    #   label3 = gr.Textbox(label="SQL query generated by LLM")

    generate.click(fn=query,inputs=input_text, outputs=label2)
demo.launch(share=True, debug=False)

/home/user/code/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://78df584d6c731181f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:I should look at the schema of the Playlist table to see what columns I can query.
Action: sql_db_schema
Action Input: Playlist
Observation: 
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/
Thought:I should look at the schema of the PlaylistTrack table to see what columns I can query.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
Pl